In [1]:
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix

import tensorflow_datasets as tfds
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os


#model.layers[4].summary()




In [2]:
displayFuncTime=1
orgSize = [512,512]
processedSize = [224,224]

def normalize(input_image, input_mask):
    input_image = tf.cast(input_image, tf.float32) / 255
    input_mask += 0
    print("normalize function input_mask:",input_mask)
    return input_image, input_mask

image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    #'label': tf.io.FixedLenFeature([], tf.int64),
    'image': tf.io.FixedLenFeature([], tf.string),
    'segmentation_mask': tf.io.FixedLenFeature([], tf.string),
    'file_name': tf.io.FixedLenFeature([], tf.string),
}
smooth = 1e-15
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + smooth) / (union + smooth)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

@tf.function
def load_image_train(datapoint):
    datapoint = tf.io.parse_single_example(datapoint, image_feature_description)
    print("load_image_train's datapoint",datapoint['image'])
    height = tf.cast(datapoint['height'], tf.int64)
    width = tf.cast(datapoint['width'], tf.int64)
    depth = tf.cast(datapoint['depth'], tf.int64)
    image = tf.io.decode_raw(datapoint['image'], tf.uint8)
    segmentation_mask = tf.io.decode_raw(datapoint['segmentation_mask'], tf.uint8)
    image = tf.reshape(image, [height,width,depth])
    segmentation_mask = tf.reshape(segmentation_mask, [height,width,1])
    print('image:',image)
    #_parsed['image'] = image
    #_parsed['segmentation_mask'] = segmentation_mask
    input_image = image
    #input_image = tf.image.resize(image, (processedSize[0], processedSize[1]))
    print("load_image_train's input_image",input_image)
    input_mask = segmentation_mask
    #input_mask = tf.image.resize(segmentation_mask, (processedSize[0], processedSize[1]))
    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)
    input_image, input_mask = normalize(input_image, input_mask)
    return input_image, input_mask

def load_image_test(datapoint):
    datapoint = tf.io.parse_single_example(datapoint, image_feature_description)
    print("load_image_test's datapoint",datapoint['image'])
    height = tf.cast(datapoint['height'], tf.int64)
    width = tf.cast(datapoint['width'], tf.int64)
    depth = tf.cast(datapoint['depth'], tf.int64)
    image = tf.io.decode_raw(datapoint['image'], tf.uint8)
    segmentation_mask = tf.io.decode_raw(datapoint['segmentation_mask'], tf.uint8)
    image = tf.reshape(image, [height,width,depth])
    segmentation_mask = tf.reshape(segmentation_mask, [height,width,1])
    print('image:',image)
    #_parsed['image'] = image
    #_parsed['segmentation_mask'] = segmentation_mask
    input_image = tf.image.resize(image, (processedSize[0], processedSize[1]))
    print("load_image_test's input_image",input_image)
    input_mask = tf.image.resize(segmentation_mask, (processedSize[0], processedSize[1]))
    input_image, input_mask = normalize(input_image, input_mask)
    return input_image, input_mask



###########


def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    #print("raw {} display list {}".format(i,display_list[i]))
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

def displayAndSave(display_list):
    plt.figure(figsize=(15, 15))
    title = ['Input Image', 'True Mask', 'Predicted Mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        #print("raw {} display list {}".format(i,display_list[i]))
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    global displayFuncTime
    plt.savefig("display-{}".format(displayFuncTime))
    displayFuncTime+=1
    plt.show()


###############

def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]
####################

np.set_printoptions(threshold=np.inf)

smooth = 1e-15

def show_predictions3(dataset=None, num=1,recordValue=False):
    if dataset:
        loopNum = 0
        for image, mask in dataset.take(num):
            #loopNum+=1
            pred_mask = model.predict(image)
            for i in image:
                loopNum+=1
                _PIL1 = np.array (tf.keras.preprocessing.image.array_to_img(mask[0]))
                _PIL2 = np.array (tf.keras.preprocessing.image.array_to_img(create_mask(pred_mask)))
                remap1 =  np.interp(_PIL1, (0, 255), (0, 1))
                remap2 =  np.interp(_PIL2, (0, 255), (0, 1))
                plt.axis('off')
                imgplot = plt.imshow(remap1+remap2)
                display([image[0], mask[0], create_mask(pred_mask)])
                print(loopNum)
                print("iou value for new dataset:",iou(remap1,remap2).numpy())
                print("dice:",dice_coef(remap1,remap2).numpy())
                iouAry.append(iou(remap1,remap2).numpy())
                diceAry.append(dice_coef(remap1,remap2).numpy())
                
                
            
            
            #_NP_ADDED = np.array(_PIL1)+np.array(_PIL2)
            #print(" remap1:",  remap1)
            
            #display([image[0], mask[0], create_mask(_NP_ADDED)])
            
            
            
    else:
        pred_mask = model.predict(sample_image[tf.newaxis, ...])
        m = iou(sample_mask, create_mask(pred_mask))
        print("iou value (sample pic):",m.numpy())
        if recordValue:
            displayAndSave([sample_image, sample_mask,create_mask(pred_mask)])
            iouHistory.append(m)
        else:
            display([sample_image, sample_mask,create_mask(pred_mask)])

            
            
def show_predictions4(dataset=None, num=1,recordValue=False):
    if dataset:
        loopNum = 0
        for image, mask in dataset.take(num):
            loopNum+=1
            pred_mask = model.predict(image)
            _PIL1 = np.array( tf.keras.preprocessing.image.array_to_img(mask[0]))
            _PIL2 = np.array (tf.keras.preprocessing.image.array_to_img(create_mask(pred_mask)))
            remap1 =  np.interp(_PIL1, (0, 255), (0, 1))
            remap2 =  np.interp(_PIL2, (0, 255), (0, 1))
            #_NP_ADDED = np.array(_PIL1)+np.array(_PIL2)
            #print(" remap1:",  remap1)
            plt.axis('off')
            imgplot = plt.imshow(remap1+remap2)
            display([image[0], mask[0], create_mask(pred_mask)])
            #display([image[0], mask[0], create_mask(_NP_ADDED)])
            print(loopNum)
            print("iou value for new dataset:",iou(remap1,remap2).numpy())
            print("dice:",dice_coef(remap1,remap2).numpy())
            iouAry.append(iou(remap1,remap2).numpy())
            diceAry.append(dice_coef(remap1,remap2).numpy())
            
            
    else:
        pred_mask = model.predict(sample_image[tf.newaxis, ...])
        m = iou(sample_mask, create_mask(pred_mask))
        print("iou value (sample pic):",m.numpy())
        if recordValue:
            displayAndSave([sample_image, sample_mask,create_mask(pred_mask)])
            iouHistory.append(m)
        else:
            display([sample_image, sample_mask,create_mask(pred_mask)])
#####################################

import numpy as np, scipy.stats as st

####################################







In [8]:
PATH=os.getcwd()
HPath="saved_model/"
Mname="043_mobileNETv2_dp2_e50_vMarcoTrainVal02052021_unAUG_bce_lr0001"
batch_size=12

model = tf.keras.models.load_model(HPath+Mname)
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
functional_1 (Functional)       [(None, 112, 112, 64 11557824    input_2[0][0]                    
__________________________________________________________________________________________________
sequential (Sequential)         (None, 14, 14, 512)  4425728     functional_1[0][4]               
__________________________________________________________________________________________________
dropout (Dropout)               (None, 14, 14, 512)  0           sequential[0][0]                 
_______________________________________________________________________________________

In [1]:
#specify the path and filename of the tfrecord dataset file to be loaded for evaluation
dname="TFrecord/test"
iouAry=[]
diceAry=[]
dataset_FULL = tf.data.TFRecordDataset(dname+'.tfrecords')
datasize =  sum(1 for _ in dataset_FULL)
#raw_image_dataset_fullSize = raw_image_dataset_fullSize.shuffle(100)
dataset_FULL_m = dataset_FULL.map(load_image_test)
dataset_FULL_b = dataset_FULL_m.batch(1)
show_predictions4(dataset_FULL_b,datasize)

m1 = np.mean(iouAry)
s1 = st.sem(iouAry)
s = st.t.interval(0.95, len(iouAry)-1, loc=np.mean(iouAry), scale=st.sem(iouAry))
m2 = np.mean(diceAry)
s2 = st.sem(diceAry)
d = st.t.interval(0.95, len(diceAry)-1, loc=np.mean(diceAry), scale=st.sem(diceAry))

print(m1)
print(s1)
print(s)


g = open("results/"+Mname+"/evaluation.txt", 'a')
print("evaluation result for {}, which has the size of {}:" .format(dname, datasize), file=g)
print("mean IOU: {}" .format(m1), file=g)
print("Standard error of mean of IOU: {}" .format(s1), file=g)
print("95% confidence interval of IOU is: {}" .format(s), file=g)
print("mean dice: {}" .format(m2), file=g)
print("Standard error of mean of dice: {}" .format(s2), file=g)
print("95% confidence interval of dice is: {}" .format(d), file=g)
g.close()

NameError: name 'tf' is not defined